03. DATA PREPARATION

In [1]:
import pandas as pd
import numpy as np

filepath = '../../data/coords/old10s-60f.csv'

df = pd.read_csv(filepath)

df.columns

Index(['frame_index', 'ball', 'gk0', 'gk1', 'id_1', 'id_2', 'id_3', 'id_4',
       'id_5', 'id_6', 'id_7', 'id_8', 'id_9', 'id_10', 'id_11', 'id_12',
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_22',
       'id_23', 'id_24', 'id_28', 'id_29', 'id_30', 'id_34', 'id_36', 'id_39',
       'id_43', 'id_44', 'id_53', 'id_61', 'id_82', 'id_83', 'id_99', 'id_105',
       'id_117', 'id_137', 'id_146', 'id_147', 'id_149', 'id_151', 'id_157',
       'id_162', 'id_163', 'id_164', 'id_165', 'id_168', 'id_169', 'id_170',
       'id_171', 'id_173', 'id_174', 'id_182', 'id_183', 'id_185', 'id_186',
       'id_188', 'id_193', 'id_196', 'id_199', 'id_200', 'id_201', 'id_204',
       'id_205'],
      dtype='object')

Create good subsets of frames

In [7]:
# Exclude the frame index column (assume it's the first column)
data_cols = df.columns[1:]

# Count non-empty (non-NaN) cells per row
df['detection_count'] = df[data_cols].notnull().sum(axis=1)

# Find indices of good frames
good_mask = df['detection_count'] > 18

# Find all consecutive sequences of good frames (no min length)
sequences = []
start = None
for idx, is_good in enumerate(good_mask):
    if is_good:
        if start is None:
            start = idx
    else:
        if start is not None:
            sequences.append((start, idx))
            start = None
# Handle sequence that goes to the end
if start is not None:
    sequences.append((start, len(df)))

# Save each sequence as a CSV
import os
os.makedirs('../../data/data_preparation/subsets', exist_ok=True)
for i, (start, end) in enumerate(sequences):
    if end - start >= 10:
        subset = df.iloc[start:end]
        subset.to_csv(f'../../data/data_preparation/subsets/subset{i+1}.csv', index=False)